In [218]:
#importing necessary libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.compose import make_column_transformer
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
import sklearn.metrics as metrics
from sklearn.linear_model import Lasso


In [219]:
# load datasets 
asmnt = pd.read_csv('C:/Users/dell/Desktop/projects/Python/assessments.csv')
crse = pd.read_csv('C:/Users/dell/Desktop/projects/Python/courses.csv')
res = pd.read_csv('C:/Users/dell/Desktop/projects/Python/studentAssessment.csv')
stud_info = pd.read_csv('C:/Users/dell/Desktop/projects/Python/studentInfo.csv')
stud_reg = pd.read_csv('C:/Users/dell/Desktop/projects/Python/studentRegistration.csv')
stud_vle = pd.read_csv('C:/Users/dell/Desktop/projects/Python/studentVle.csv')
vle = pd.read_csv('C:/Users/dell/Desktop/projects/Python/vle.csv') # VLE = virtual learning environment


# Overview of the steps to be performed in this project

1. data cleaning of different tables on an individual level, we have total of 7 different tables,
that will be merged in the end
2. merging the different tables after making necessary changes to the data(taken help from kaggle)
3. Feature engineering 
4. running classification and regression models on relevant variables, displaying performance metrics 

# .


# 1. data cleaning of different tables on an individual level, we have total of 7 different tables, that will be merged in the end

# Table 1. courses.csv

### File contains the list of all available modules and their presentations. 

### The columns are:

#### • code_module – code name of the module, which serves as the identifier.
#### • code_presentation – code name of the presentation. It consists of the year and “B” for the presentation starting in February and “J” for the presentation starting in October.
#### • length - length of the module-presentation in days.

The structure of B and J presentations may differ and therefore it is good practice to analyse
the B and J presentations separately. Nevertheless, for some presentations the
corresponding previous B/J presentation do not exist and therefore the J presentation must
be used to inform the B presentation or vice versa. In the dataset this is the case of CCC, EEE
and GGG modules

In [220]:
crse.head()

,code_module,code_presentation,module_presentation_length
0,AAA,2013J,268
1,AAA,2014J,269
2,BBB,2013J,268
3,BBB,2014J,262
4,BBB,2013B,240


In [221]:
# all variables are correctly classified to their respective data types and we have no null values
crse.info()
print(crse.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 3 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   code_module                 22 non-null     object
 1   code_presentation           22 non-null     object
 2   module_presentation_length  22 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 656.0+ bytes
code_module                   0
code_presentation             0
module_presentation_length    0
dtype: int64


In [273]:
# below means we have 7 unique course module, 
crse.nunique()
# crse[crse.duplicated()]

code_module                   7
code_presentation             4
module_presentation_length    7
dtype: int64

In [272]:
crse.describe()

,module_presentation_length
count,22.000000
mean,255.545455
std,13.654677
min,234.000000
25%,241.000000
50%,261.500000
75%,268.000000
max,269.000000


# Table 2. assessments.csv
### This file contains information about assessments in module-presentations. Usually, every presentation has a number of assessments followed by the final exam. CSV contains columns:

#### • code_module – identification code of the module, to which the assessment belongs.
#### • code_presentation - identification code of the presentation, to which the assessment belongs.
#### • id_assessment – identification number of the assessment.
#### • assessment_type – type of assessment. Three types of assessments exist: Tutor Marked Assessment (TMA), Computer Marked Assessment (CMA) and Final Exam (Exam).
#### • date – information about the final submission date of the assessment calculated as the number of days since the start of the module-presentation. The starting date of the presentation has number 0 (zero).
#### • weight - weight of the assessment in %. Typically, Exams are treated separately and have the weight 100%; the sum of all other assessments is 100%. 

If the information about the final exam date is missing, it is at the end of the last presentation week.

In [224]:
asmnt.head()


,code_module,code_presentation,id_assessment,assessment_type,date,weight
0,AAA,2013J,1752,TMA,19.0,10.0
1,AAA,2013J,1753,TMA,54.0,20.0
2,AAA,2013J,1754,TMA,117.0,20.0
3,AAA,2013J,1755,TMA,166.0,20.0
4,AAA,2013J,1756,TMA,215.0,30.0


In [225]:
#below shows we 3 assessment types
asmnt.nunique()

code_module            7
code_presentation      4
id_assessment        206
assessment_type        3
date                  74
weight                24
dtype: int64

In [226]:
print(asmnt.info())
print(asmnt.isnull().sum())

# since ID number may not be a int type, we will convert into object type to remove confusion
asmnt['id_assessment'] = asmnt['id_assessment'].astype(object)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206 entries, 0 to 205
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   code_module        206 non-null    object 
 1   code_presentation  206 non-null    object 
 2   id_assessment      206 non-null    int64  
 3   assessment_type    206 non-null    object 
 4   date               195 non-null    float64
 5   weight             206 non-null    float64
dtypes: float64(2), int64(1), object(3)
memory usage: 9.8+ KB
None
code_module           0
code_presentation     0
id_assessment         0
assessment_type       0
date                 11
weight                0
dtype: int64


In [227]:
asmnt[asmnt['date'].isnull()]

,code_module,code_presentation,id_assessment,assessment_type,date,weight
5,AAA,2013J,1757,Exam,NaN,100.0
11,AAA,2014J,1763,Exam,NaN,100.0
23,BBB,2013B,14990,Exam,NaN,100.0
35,BBB,2013J,15002,Exam,NaN,100.0
47,BBB,2014B,15014,Exam,NaN,100.0
53,BBB,2014J,15025,Exam,NaN,100.0
62,CCC,2014B,24290,Exam,NaN,100.0
63,CCC,2014B,40087,Exam,NaN,100.0
72,CCC,2014J,24299,Exam,NaN,100.0
73,CCC,2014J,40088,Exam,NaN,100.0


In [228]:
# Using group by to analyse, we see that in many places TMA and CMA have low weightage, specially CMA(CMA is an assessment type)

asmnt.groupby(['code_module','code_presentation',"assessment_type"]).agg(total_weight = ('weight',sum))

total_weight
code_module code_presentation assessment_type              
AAA         2013J             Exam                    100.0
                              TMA                     100.0
            2014J             Exam                    100.0
                              TMA                     100.0
BBB         2013B             CMA                       5.0
                              Exam                    100.0
                              TMA                      95.0
            2013J             CMA                       5.0
                              Exam                    100.0
                              TMA                      95.0
            2014B             CMA                       5.0
                              Exam                    100.0
                              TMA                      95.0
            2014J             Exam                    100.0
                              TMA                     100.0
CCC         2014B             CMA                      25.0
                              Exam                    200.0
                              TMA                      75.0
            2014J             CMA                      25.0
                              Exam                    200.0
                              TMA                      75.0
DDD         2013B             CMA                      25.0
                              Exam                    100.0
                              TMA                      75.0
            2013J             Exam                    100.0
                              TMA                     100.0
            2014B             Exam                    100.0
                              TMA                     100.0
            2014J             Exam                    100.0
                              TMA                     100.0
EEE         2013J             Exam                    100.0
                              TMA                     100.0
            2014B             Exam                    100.0
                              TMA                     100.0
            2014J             Exam                    100.0
                              TMA                     100.0
FFF         2013B             CMA                       0.0
                              Exam                    100.0
                              TMA                     100.0
            2013J             CMA                       0.0
                              Exam                    100.0
                              TMA                     100.0
            2014B             CMA                       0.0
                              Exam                    100.0
                              TMA                     100.0
            2014J             CMA                       0.0
                              Exam                    100.0
                              TMA                     100.0
GGG         2013J             CMA                       0.0
                              Exam                    100.0
                              TMA                       0.0
            2014B             CMA                       0.0
                              Exam                    100.0
                              TMA                       0.0
            2014J             CMA                       0.0
                              Exam                    100.0
                              TMA                       0.0

# 3. vle.csv
### The csv file contains information about the available materials in the VLE. Typically these are html pages, pdf files, etc. Students have access to these materials online and their interactions with the materials are recorded. The vle.csv file contains the following columns:

#### • id_site – an identification number of the material.
#### • code_module – an identification code for module.
#### • code_presentation - the identification code of presentation.
#### • activity_type – the role associated with the module material.
#### • week_from – the week from which the material is planned to be used.
#### • week_to – week until which the material is planned to be used.


In [229]:
vle.head()

,id_site,code_module,code_presentation,activity_type,week_from,week_to
0,546943,AAA,2013J,resource,NaN,NaN
1,546712,AAA,2013J,oucontent,NaN,NaN
2,546998,AAA,2013J,resource,NaN,NaN
3,546888,AAA,2013J,url,NaN,NaN
4,547035,AAA,2013J,resource,NaN,NaN


In [230]:
vle.nunique()

id_site              6364
code_module             7
code_presentation       4
activity_type          20
week_from              30
week_to                30
dtype: int64

In [231]:
# week from / week to consists of large number of nulls and id_site has been turned into an object data type 

print(vle.info())
print(vle.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6364 entries, 0 to 6363
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id_site            6364 non-null   int64  
 1   code_module        6364 non-null   object 
 2   code_presentation  6364 non-null   object 
 3   activity_type      6364 non-null   object 
 4   week_from          1121 non-null   float64
 5   week_to            1121 non-null   float64
dtypes: float64(2), int64(1), object(3)
memory usage: 298.4+ KB
None
id_site                 0
code_module             0
code_presentation       0
activity_type           0
week_from            5243
week_to              5243
dtype: int64


In [232]:
vle['id_site']  =  vle['id_site'].astype(object)

# 4. studentInfo.csv
### This file contains demographic information about the students together with their results. File contains the following columns:
#### • code_module – an identification code for a module on which the student is registered.
#### • code_presentation - the identification code of the presentation during which the student is registered on the module.
#### • id_student – a unique identification number for the student.
#### • gender – the student’s gender.
#### • region – identifies the geographic region, where the student lived while taking the module-presentation.
#### • highest_education – highest student education level on entry to the module presentation.
#### • imd_band – specifies the Index of Multiple Depravation band of the place where the student lived during the module-presentation.
#### • age_band – band of the student’s age.
#### • num_of_prev_attempts – the number times the student has attempted this module.
#### • studied_credits – the total number of credits for the modules the student is currently studying.
#### • disability – indicates whether the student has declared a disability.
#### • final_result – student’s final result in the module-presentation.


In [233]:
stud_info.head()

,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result
0,AAA,2013J,11391,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass
1,AAA,2013J,28400,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass
2,AAA,2013J,30268,F,North Western Region,A Level or Equivalent,30-40%,35-55,0,60,Y,Withdrawn
3,AAA,2013J,31604,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,Pass
4,AAA,2013J,32885,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,Pass


In [234]:
# we have a total 28,785 unique student id or students data with us,
stud_info.nunique()

code_module                 7
code_presentation           4
id_student              28785
gender                      2
region                     13
highest_education           5
imd_band                   10
age_band                    3
num_of_prev_attempts        7
studied_credits            61
disability                  2
final_result                4
dtype: int64

In [235]:
print(stud_info.info())
print(stud_info.isnull().sum())

stud_info['id_student'] = stud_info['id_student'].astype(object)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32593 entries, 0 to 32592
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   code_module           32593 non-null  object
 1   code_presentation     32593 non-null  object
 2   id_student            32593 non-null  int64 
 3   gender                32593 non-null  object
 4   region                32593 non-null  object
 5   highest_education     32593 non-null  object
 6   imd_band              31482 non-null  object
 7   age_band              32593 non-null  object
 8   num_of_prev_attempts  32593 non-null  int64 
 9   studied_credits       32593 non-null  int64 
 10  disability            32593 non-null  object
 11  final_result          32593 non-null  object
dtypes: int64(3), object(9)
memory usage: 3.0+ MB
None
code_module                0
code_presentation          0
id_student                 0
gender                     0
region              

# 5. studentRegistration.csv
### This file contains information about the time when the student registered for the module presentation. For students who unregistered the date of unregistration is also recorded. File contains five columns:
#### • code_module – an identification code for a module.
#### • code_presentation - the identification code of the presentation.
#### • id_student – a unique identification number for the student.
#### • date_registration – the date of student’s registration on the module presentation,this is the number of days measured relative to the start of the module-presentation (e.g. the negative value -30 means that the student registered to module presentation 30 days before it started).
#### • date_unregistration – date of student unregistration from the module presentation,this is the number of days measured relative to the start of the modulepresentation. Students, who completed the course have this field empty. Students who unregistered have Withdrawal as the value of the final_result column in the studentInfo.csv file.

In [236]:
stud_reg.head()

,code_module,code_presentation,id_student,date_registration,date_unregistration
0,AAA,2013J,11391,-159.0,NaN
1,AAA,2013J,28400,-53.0,NaN
2,AAA,2013J,30268,-92.0,12.0
3,AAA,2013J,31604,-52.0,NaN
4,AAA,2013J,32885,-176.0,NaN


In [237]:
stud_reg.nunique()

code_module                7
code_presentation          4
id_student             28785
date_registration        332
date_unregistration      416
dtype: int64

In [238]:
print(stud_reg.info())
print(stud_reg.isnull().sum())

stud_reg['id_student'] = stud_reg['id_student'].astype(object)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32593 entries, 0 to 32592
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   code_module          32593 non-null  object 
 1   code_presentation    32593 non-null  object 
 2   id_student           32593 non-null  int64  
 3   date_registration    32548 non-null  float64
 4   date_unregistration  10072 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 1.2+ MB
None
code_module                0
code_presentation          0
id_student                 0
date_registration         45
date_unregistration    22521
dtype: int64


# 6. studentAssessment.csv
### This file contains the results of students’ assessments. If the student does not submit the assessment, no result is recorded. The final exam submissions is missing, if the result of the assessments is not stored in the system. This file contains the following columns:
#### • id_assessment – the identification number of the assessment.
#### • id_student – a unique identification number for the student.
#### • date_submitted – the date of student submission, measured as the number of days since the start of the module presentation.
#### • is_banked – a status flag indicating that the assessment result has been transferredfrom a previous presentation.
#### • score – the student’s score in this assessment. The range is from 0 to 100. The score lower than 40 is interpreted as Fail. The marks are in the range from 0 to 100.

In [239]:
res.head()

,id_assessment,id_student,date_submitted,is_banked,score
0,1752,11391,18,0,78.0
1,1752,28400,22,0,70.0
2,1752,31604,17,0,72.0
3,1752,32885,26,0,69.0
4,1752,38053,19,0,79.0


In [240]:
res.nunique()

id_assessment       188
id_student        23369
date_submitted      312
is_banked             2
score               101
dtype: int64

In [241]:
print(res.info())
print(res.isnull().sum())

res['id_assessment'] = res['id_assessment'].astype(object)
res['id_student'] = res['id_student'].astype(object)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173912 entries, 0 to 173911
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id_assessment   173912 non-null  int64  
 1   id_student      173912 non-null  int64  
 2   date_submitted  173912 non-null  int64  
 3   is_banked       173912 non-null  int64  
 4   score           173739 non-null  float64
dtypes: float64(1), int64(4)
memory usage: 6.6 MB
None
id_assessment       0
id_student          0
date_submitted      0
is_banked           0
score             173
dtype: int64


In [242]:
# assuming score columns in results when null refers to a 0 score
res.fillna(0, inplace=True)

# Table 7. studentVle.csv
### The studentVle.csv file contains information about each student’s interactions with the materials in the VLE. This file contains the following columns:
#### • code_module – an identification code for a module.
#### • code_presentation - the identification code of the module presentation.
#### • id_student – a unique identification number for the student.
#### • id_site - an identification number for the VLE material.
#### • date – the date of student’s interaction with the material measured as the number of days since the start of the module-presentation.
#### • sum_click – the number of times a student interacts with the material in that day

In [243]:
stud_vle.head()

,code_module,code_presentation,id_student,id_site,date,sum_click
0,AAA,2013J,28400,546652,-10,4
1,AAA,2013J,28400,546652,-10,1
2,AAA,2013J,28400,546652,-10,1
3,AAA,2013J,28400,546614,-10,11
4,AAA,2013J,28400,546714,-10,1


In [244]:
stud_vle.nunique()

code_module              7
code_presentation        4
id_student           26074
id_site               6268
date                   295
sum_click              498
dtype: int64

In [245]:
print(stud_vle.info())
print(stud_vle.isnull().sum())

stud_vle['id_student'] = stud_vle['id_student'].astype(object)
stud_vle['id_site'] = stud_vle['id_site'].astype(object)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10655280 entries, 0 to 10655279
Data columns (total 6 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   code_module        object
 1   code_presentation  object
 2   id_student         int64 
 3   id_site            int64 
 4   date               int64 
 5   sum_click          int64 
dtypes: int64(4), object(2)
memory usage: 487.8+ MB
None
code_module          0
code_presentation    0
id_student           0
id_site              0
date                 0
sum_click            0
dtype: int64


# 2. Merging the different tables after making necessary changes to the data(taken help from kaggle)

# Merging above tables using the schema provided by the data publishers

#### for merging the data, kaggle sources were used, as merging is not the main goal of this assignment and merging 7 tables was a very big and messy task ****

In [246]:
#useful when merging findDiffValues(materials, vle, 'id_site'), compareCols(materials, vle)


In [247]:
# Merge with an inner join
studvle_vle = pd.merge(stud_vle,vle, on=['code_module', 'code_presentation', 'id_site'], how='inner')
# Drop columns
studvle_vle.drop(columns=['week_from', 'week_to', 'date'], inplace=True)

studvle_vle.head()

,code_module,code_presentation,id_student,id_site,sum_click,activity_type
0,AAA,2013J,28400,546652,4,forumng
1,AAA,2013J,28400,546652,1,forumng
2,AAA,2013J,28400,546652,1,forumng
3,AAA,2013J,28400,546652,8,forumng
4,AAA,2013J,30268,546652,3,forumng


In [248]:
# adjusting the joined table to our requirements, below we can see how many clicks students did on VLE(Virtual learning environment)

click_vle = studvle_vle.groupby(['code_module', 'code_presentation', 'id_student']).agg(total_click = ("sum_click",sum)).reset_index()

click_vle.head()

,code_module,code_presentation,id_student,total_click
0,AAA,2013J,11391,934
1,AAA,2013J,28400,1435
2,AAA,2013J,30268,281
3,AAA,2013J,31604,2158
4,AAA,2013J,32885,1034


In [249]:
# Merge with an inner join
studreg_crse = pd.merge(stud_reg, crse, on=['code_module', 'code_presentation'], how='inner')

print(studreg_crse.head())

# Merge with an inner join
studreg_crse_info = pd.merge(studreg_crse,stud_info,  on=['code_module', 'code_presentation', 'id_student'], how='inner')

studreg_crse_info.head()

  code_module code_presentation id_student  date_registration  \
0         AAA             2013J      11391             -159.0   
1         AAA             2013J      28400              -53.0   
2         AAA             2013J      30268              -92.0   
3         AAA             2013J      31604              -52.0   
4         AAA             2013J      32885             -176.0   

   date_unregistration  module_presentation_length  
0                  NaN                         268  
1                  NaN                         268  
2                 12.0                         268  
3                  NaN                         268  
4                  NaN                         268  


,code_module,code_presentation,id_student,date_registration,date_unregistration,module_presentation_length,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result
0,AAA,2013J,11391,-159.0,NaN,268,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass
1,AAA,2013J,28400,-53.0,NaN,268,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass
2,AAA,2013J,30268,-92.0,12.0,268,F,North Western Region,A Level or Equivalent,30-40%,35-55,0,60,Y,Withdrawn
3,AAA,2013J,31604,-52.0,NaN,268,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,Pass
4,AAA,2013J,32885,-176.0,NaN,268,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,Pass


In [250]:
# merge with an inner join
asmnt_res = pd.merge(asmnt, res, on=['id_assessment'], how='inner')
# Rearrange column names
asmnt_res = asmnt_res[['id_student', 'code_module', 'code_presentation', 'id_assessment', 'assessment_type', 'date', 'date_submitted', 'weight', 'is_banked', 'score']]

asmnt_res.head()

,id_student,code_module,code_presentation,id_assessment,assessment_type,date,date_submitted,weight,is_banked,score
0,11391,AAA,2013J,1752,TMA,19.0,18,10.0,0,78.0
1,28400,AAA,2013J,1752,TMA,19.0,22,10.0,0,70.0
2,31604,AAA,2013J,1752,TMA,19.0,17,10.0,0,72.0
3,32885,AAA,2013J,1752,TMA,19.0,26,10.0,0,69.0
4,38053,AAA,2013J,1752,TMA,19.0,19,10.0,0,79.0


In [251]:
# Make a copy of dataset
scores = asmnt_res

# Count how many exams there are in Results for every module presentation
scores[scores['assessment_type'] == 'Exam'][['code_module', 'code_presentation', 'id_assessment']]\
.groupby(['code_module', 'code_presentation'])\
.nunique()


id_assessment
code_module code_presentation               
CCC         2014B                          1
            2014J                          1
DDD         2013B                          1
            2013J                          1
            2014B                          1
            2014J                          1

In [252]:
### Make helper columns ###
# (a) Add column multiplying weight and score
scores['weight*score'] = scores['weight']*scores['score']
# (b) Aggregate recorded weight*score per student
    # per module presentation
sum_scores = scores\
.groupby(['id_student', 'code_module', 'code_presentation'])\
.agg(weightByScore = ('weight*score', sum))\
.reset_index()
# (c) Calculate total recorded weight of module
# (c.i) Get total weight of modules
total_weight = asmnt\
.groupby(['code_module', 'code_presentation'])\
.agg(total_weight = ('weight', sum))\
.reset_index()
# (c.ii) Subtract 100 to account for missing exams
total_weight['total_weight'] = total_weight['total_weight']-100
# (c.iii) Mark module DDD as having 200 credits 
total_weight.loc[(total_weight.code_module == 'DDD'), 'total_weight'] = 200

### Calculate weighted score ###
# (a) Merge sum_scores and total_weight tables
score_weights = pd.merge(sum_scores, total_weight, on=['code_module', 'code_presentation'], how='inner')
# (b) Calculate weighted score
score_weights['weighted_score'] = score_weights['weightByScore'] / score_weights['total_weight']
# (c) Drop helper columns
score_weights.drop(columns=['weightByScore', 'total_weight'], inplace=True)

In [253]:
score_weights.head()


,id_student,code_module,code_presentation,weighted_score
0,6516,AAA,2014J,63.5
1,24734,AAA,2014J,47.5
2,26192,AAA,2014J,86.3
3,28061,AAA,2014J,77.4
4,31600,AAA,2014J,55.5


In [254]:
# Calculate the difference between the submission dates
lateSubmission = asmnt_res.assign(submission_days=asmnt_res['date_submitted']-asmnt_res['date'])
# Make a column indicating if the submission was late or not 
lateSubmission = lateSubmission.assign(late_submission=lateSubmission['submission_days'] > 0)

lateSubmission.head()

,id_student,code_module,code_presentation,id_assessment,assessment_type,date,date_submitted,weight,is_banked,score,weight*score,submission_days,late_submission
0,11391,AAA,2013J,1752,TMA,19.0,18,10.0,0,78.0,780.0,-1.0,False
1,28400,AAA,2013J,1752,TMA,19.0,22,10.0,0,70.0,700.0,3.0,True
2,31604,AAA,2013J,1752,TMA,19.0,17,10.0,0,72.0,720.0,-2.0,False
3,32885,AAA,2013J,1752,TMA,19.0,26,10.0,0,69.0,690.0,7.0,True
4,38053,AAA,2013J,1752,TMA,19.0,19,10.0,0,79.0,790.0,0.0,False


In [255]:
lateSubmission[(lateSubmission['assessment_type'] == 'Exam') & (lateSubmission['late_submission'] == True)]

,id_student,code_module,code_presentation,id_assessment,assessment_type,date,date_submitted,weight,is_banked,score,weight*score,submission_days,late_submission
74900,81351,DDD,2013B,25340,Exam,240.0,248,100.0,0,0.0,0.0,8.0,True
75194,532491,DDD,2013B,25340,Exam,240.0,241,100.0,0,49.0,4900.0,1.0,True
75221,538611,DDD,2013B,25340,Exam,240.0,241,100.0,0,51.0,5100.0,1.0,True
75282,548535,DDD,2013B,25340,Exam,240.0,241,100.0,0,60.0,6000.0,1.0,True
87510,391924,DDD,2014B,25361,Exam,241.0,251,100.0,0,69.0,6900.0,10.0,True
87861,631388,DDD,2014B,25361,Exam,241.0,242,100.0,0,78.0,7800.0,1.0,True
87893,634383,DDD,2014B,25361,Exam,241.0,242,100.0,0,96.0,9600.0,1.0,True


In [256]:
# Aggregate per student per module presentation
total_late_per_student = lateSubmission\
.groupby(['id_student', 'code_module', 'code_presentation'])\
.agg(total_late_submission = ('late_submission', sum))\
.reset_index()

total_late_per_student.head()

,id_student,code_module,code_presentation,total_late_submission
0,6516,AAA,2014J,0
1,8462,DDD,2013J,1
2,8462,DDD,2014J,0
3,11391,AAA,2013J,0
4,23629,BBB,2013B,3


In [257]:
# Make a df with total number of all assessments per student per module presentation
total_count_assessments = lateSubmission[['id_student', 'code_module', 'code_presentation', 'id_assessment']]\
.groupby(['id_student', 'code_module', 'code_presentation'])\
.size()\
.reset_index(name='total_assessments')

total_count_assessments.head()

,id_student,code_module,code_presentation,total_assessments
0,6516,AAA,2014J,5
1,8462,DDD,2013J,3
2,8462,DDD,2014J,4
3,11391,AAA,2013J,5
4,23629,BBB,2013B,4


In [258]:
# Merge df with total late assessements and total count assessments
late_rate_per_student = pd.merge(total_late_per_student, total_count_assessments, on=['id_student', 'code_module', 'code_presentation'], how='inner')
# Make a new column with late submission rate
late_rate_per_student['late_rate'] = late_rate_per_student['total_late_submission'] / late_rate_per_student['total_assessments']
# Drop helper columns
late_rate_per_student.drop(columns=['total_late_submission', 'total_assessments'], inplace=True)

late_rate_per_student

,id_student,code_module,code_presentation,late_rate
0,6516,AAA,2014J,0.000000
1,8462,DDD,2013J,0.333333
2,8462,DDD,2014J,0.000000
3,11391,AAA,2013J,0.000000
4,23629,BBB,2013B,0.750000
...,...,...,...,...
25838,2698257,AAA,2013J,0.400000
25839,2698535,CCC,2014B,0.750000
25840,2698535,EEE,2013J,0.250000
25841,2698577,BBB,2014J,0.000000


In [259]:
# Define function for marking failed assignments
passRate = asmnt_res
passRate = passRate.assign(fail=passRate['score'] < 40)

passRate.head()

,id_student,code_module,code_presentation,id_assessment,assessment_type,date,date_submitted,weight,is_banked,score,weight*score,fail
0,11391,AAA,2013J,1752,TMA,19.0,18,10.0,0,78.0,780.0,False
1,28400,AAA,2013J,1752,TMA,19.0,22,10.0,0,70.0,700.0,False
2,31604,AAA,2013J,1752,TMA,19.0,17,10.0,0,72.0,720.0,False
3,32885,AAA,2013J,1752,TMA,19.0,26,10.0,0,69.0,690.0,False
4,38053,AAA,2013J,1752,TMA,19.0,19,10.0,0,79.0,790.0,False


In [260]:
# Aggregate per student per module presentation
total_fails_per_student = passRate\
.groupby(['id_student', 'code_module', 'code_presentation'])\
.agg(total_fails = ("fail",sum))\
.reset_index()

total_fails_per_student.head()

,id_student,code_module,code_presentation,total_fails
0,6516,AAA,2014J,0
1,8462,DDD,2013J,0
2,8462,DDD,2014J,0
3,11391,AAA,2013J,0
4,23629,BBB,2013B,0


In [261]:
# Merge df with total fails and total count assessments
fail_rate_per_student = pd.merge(total_fails_per_student, total_count_assessments, on=['id_student', 'code_module', 'code_presentation'], how='inner')
# Make a new column with late submission rate
fail_rate_per_student['fail_rate'] = fail_rate_per_student['total_fails'] / fail_rate_per_student['total_assessments']
# Drop helper columns
fail_rate_per_student.drop(columns=['total_fails', 'total_assessments'], inplace=True)

fail_rate_per_student

,id_student,code_module,code_presentation,fail_rate
0,6516,AAA,2014J,0.00
1,8462,DDD,2013J,0.00
2,8462,DDD,2014J,0.00
3,11391,AAA,2013J,0.00
4,23629,BBB,2013B,0.00
...,...,...,...,...
25838,2698257,AAA,2013J,0.00
25839,2698535,CCC,2014B,0.75
25840,2698535,EEE,2013J,0.25
25841,2698577,BBB,2014J,0.00


In [262]:
assessments = pd.merge(score_weights, late_rate_per_student, on=['id_student', 'code_module', 'code_presentation'], how='inner')
assessments = pd.merge(assessments, fail_rate_per_student, on=['id_student', 'code_module', 'code_presentation'], how='inner')

assessments.head()

,id_student,code_module,code_presentation,weighted_score,late_rate,fail_rate
0,6516,AAA,2014J,63.5,0.0,0.0
1,24734,AAA,2014J,47.5,0.8,0.2
2,26192,AAA,2014J,86.3,0.0,0.0
3,28061,AAA,2014J,77.4,0.2,0.0
4,31600,AAA,2014J,55.5,0.6,0.0


In [263]:
merged = pd.merge(studreg_crse_info, click_vle, on=['id_student', 'code_module', 'code_presentation'], how='left')

merged.head()

,code_module,code_presentation,id_student,date_registration,date_unregistration,module_presentation_length,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result,total_click
0,AAA,2013J,11391,-159.0,NaN,268,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass,934.0
1,AAA,2013J,28400,-53.0,NaN,268,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass,1435.0
2,AAA,2013J,30268,-92.0,12.0,268,F,North Western Region,A Level or Equivalent,30-40%,35-55,0,60,Y,Withdrawn,281.0
3,AAA,2013J,31604,-52.0,NaN,268,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,Pass,2158.0
4,AAA,2013J,32885,-176.0,NaN,268,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,Pass,1034.0


In [264]:
merged = pd.merge(merged, assessments, on=['id_student', 'code_module', 'code_presentation'], how='left')

merged.head()

,code_module,code_presentation,id_student,date_registration,date_unregistration,module_presentation_length,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result,total_click,weighted_score,late_rate,fail_rate
0,AAA,2013J,11391,-159.0,NaN,268,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass,934.0,82.4,0.0,0.0
1,AAA,2013J,28400,-53.0,NaN,268,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass,1435.0,65.4,0.4,0.0
2,AAA,2013J,30268,-92.0,12.0,268,F,North Western Region,A Level or Equivalent,30-40%,35-55,0,60,Y,Withdrawn,281.0,NaN,NaN,NaN
3,AAA,2013J,31604,-52.0,NaN,268,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,Pass,2158.0,76.3,0.0,0.0
4,AAA,2013J,32885,-176.0,NaN,268,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,Pass,1034.0,55.0,1.0,0.4


In [265]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32593 entries, 0 to 32592
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   code_module                 32593 non-null  object 
 1   code_presentation           32593 non-null  object 
 2   id_student                  32593 non-null  object 
 3   date_registration           32548 non-null  float64
 4   date_unregistration         10072 non-null  float64
 5   module_presentation_length  32593 non-null  int64  
 6   gender                      32593 non-null  object 
 7   region                      32593 non-null  object 
 8   highest_education           32593 non-null  object 
 9   imd_band                    31482 non-null  object 
 10  age_band                    32593 non-null  object 
 11  num_of_prev_attempts        32593 non-null  int64  
 12  studied_credits             32593 non-null  int64  
 13  disability                  325

In [266]:
#since many columns have null, final data cleaning needs to be done 
merged.isnull().sum()

code_module                       0
code_presentation                 0
id_student                        0
date_registration                45
date_unregistration           22521
module_presentation_length        0
gender                            0
region                            0
highest_education                 0
imd_band                       1111
age_band                          0
num_of_prev_attempts              0
studied_credits                   0
disability                        0
final_result                      0
total_click                    3365
weighted_score                 8862
late_rate                      6750
fail_rate                      6750
dtype: int64

In [113]:
# Find what is the most frequent band in each region
regions_list = list(merged\
                    [merged['imd_band'].isnull()]['region']\
                    .unique())
print(regions_list)

for i in regions_list:
    merged['imd_band'] = np.where( ( (merged['imd_band'].isnull()) & (merged['region'] == i) ),
                                           merged[merged['region'] == i].imd_band.mode(),
                                           merged['imd_band']
                                    )

['North Region',
 'Ireland',
 'West Midlands Region',
 'South Region',
 'South West Region',
 'North Western Region',
 'Yorkshire Region',
 'Scotland']

In [128]:
merged['date_registration'] = np.where( (merged['date_registration'].isnull()),
                                           merged.date_registration.median(),
                                           merged['date_registration']
                                    )

In [1]:
# merged.date_registration.median()

In [130]:
merged['total_click'] = merged['total_click'].replace(np.nan).fillna(0)

In [132]:
merged['fail_rate'] = merged['fail_rate'].replace(np.nan).fillna(1.0)

In [133]:
merged['late_rate'] = merged['late_rate'].replace(np.nan).fillna(1.0)

In [131]:
merged['weighted_score'] = merged['weighted_score'].replace(np.nan).fillna(0)

In [135]:
# Drop unneeded columns
merged.drop(columns=['id_student'], inplace=True)# no more needed since we have done the merging of the data now
merged.drop(columns=['final_result'], inplace=True) #removing because we will be predicting weighted scores
merged.drop(columns=['date_unregistration'], inplace=True)# it has majority of the data as null and should be dropped


In [136]:
merged.head()

,code_module,code_presentation,date_registration,module_presentation_length,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,total_click,weighted_score,late_rate,fail_rate
0,AAA,2013J,-159.0,268,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,934.0,82.4,0.0,0.0
1,AAA,2013J,-53.0,268,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,1435.0,65.4,0.4,0.0
2,AAA,2013J,-92.0,268,F,North Western Region,A Level or Equivalent,30-40%,35-55,0,60,Y,281.0,65.4,0.4,0.0
3,AAA,2013J,-52.0,268,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,2158.0,76.3,0.0,0.0
4,AAA,2013J,-176.0,268,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,1034.0,55.0,1.0,0.4


In [138]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32593 entries, 0 to 32592
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   code_module                 32593 non-null  object 
 1   code_presentation           32593 non-null  object 
 2   date_registration           32593 non-null  float64
 3   module_presentation_length  32593 non-null  int64  
 4   gender                      32593 non-null  object 
 5   region                      32593 non-null  object 
 6   highest_education           32593 non-null  object 
 7   imd_band                    32593 non-null  object 
 8   age_band                    32593 non-null  object 
 9   num_of_prev_attempts        32593 non-null  int64  
 10  studied_credits             32593 non-null  int64  
 11  disability                  32593 non-null  object 
 12  total_click                 32593 non-null  float64
 13  weighted_score              325

In [157]:
merged_copy = merged

In [178]:
# # define ordinal encoding
# oe = OrdinalEncoder()

# #Unfortunately outputs an array instead of dataframe.
# array_ordi_encoded = oe.fit_transform(merged[['highest_education', 'imd_band']])


In [179]:
# #Convert it to df
# data_ordi_encoded = pd.DataFrame(array_ordi_encoded, index=merged.index)

# #list of columns not needed now, where OHE was applied
# cols_non_oe = list(set(merged.columns) - set(['highest_education', 'imd_band']))

# #Extract only the columns that didnt need to be encoded
# data_other_cols = merged.drop(columns= ['highest_education', 'imd_band'])

# #Concatenate the two dataframes : 
# data_out1 = pd.concat([data_ordi_encoded, merged[cols_non_oe]], axis=1)

In [180]:
# data_out1

In [181]:

# # define one hot encoding
# ohe = OneHotEncoder()
# # transform data

# #One-hot-encode the categorical columns.
# #Unfortunately outputs an array instead of dataframe.
# array_hot_encoded = ohe.fit_transform(data_out1[['code_module', 'code_presentation', 'gender', 'region', 'age_band', 'disability']])



In [182]:
# #Convert it to df
# data_hot_encoded = pd.DataFrame(array_hot_encoded, index=data_out1.index)

# #list of columns not needed now, where OHE was applied
# cols_non_ohe = list(set(data_out1.columns) - set(['code_module', 'code_presentation', 'gender', 'region', 'age_band', 'disability']))

# #Extract only the columns that didnt need to be encoded
# data_other_cols = data_out1.drop(columns= ['code_module', 'code_presentation', 'gender', 'region', 'age_band', 'disability'])

# #Concatenate the two dataframes : 
# data_out = pd.concat([data_hot_encoded, data_other_cols], axis=1)

In [183]:
# data_out

In [189]:
# list(set(merged.columns) - set(['code_module', 'code_presentation', 'gender', 'region', 'age_band', 'disability']))

In [192]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(merged, test_size=0.2, random_state=42, stratify=merged['code_module'])

In [194]:
X_train = train.drop(columns=[ 'weighted_score'])
y_train = train['weighted_score']
X_test = test.drop(columns=['weighted_score'])
y_test =  test['weighted_score'].copy()

In [195]:
# Set encoding and scaling instructions
column_transform = make_column_transformer(
    (OneHotEncoder(), ['code_module', 'code_presentation', 'gender', 'region', 'age_band', 'disability']),
    (OrdinalEncoder(), ['highest_education', 'imd_band']),
    (RobustScaler(), ['date_registration', 'module_presentation_length',
                       'num_of_prev_attempts', 'studied_credits', 'total_click', 'late_rate',
                       'fail_rate'])
)

# Apply column transformer to features
X_trans = column_transform.fit_transform(X_train)

In [198]:
pd.DataFrame(X_trans)

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,5.0,0.291667,-0.777778,1.0,0.166667,0.647393,-0.114286,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,2.0,0.347222,-0.037037,0.0,1.000000,-0.022393,-0.400000,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,7.0,0.500000,0.259259,0.0,0.000000,0.827206,1.100000,0.0
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,2.0,1.0,-0.930556,0.259259,0.0,0.000000,0.944184,-0.036364,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,2.0,1.0,-0.791667,0.222222,0.0,0.000000,-0.359291,-0.400000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26069,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,2.0,-1.166667,0.259259,0.0,0.000000,2.979612,-0.400000,0.0
26070,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,3.0,-0.097222,0.000000,0.0,1.000000,-0.456885,-0.400000,0.2
26071,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,2.0,0.0,-0.930556,-0.814815,2.0,0.000000,-0.468249,1.600000,0.0
26072,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,2.0,9.0,-1.555556,-0.037037,0.0,0.000000,0.678142,-0.400000,0.0


In [203]:
# Setting up the pipeline
lm = LinearRegression()

lm_pipeline = make_pipeline(column_transform, lm)

# Fit the training data
lm_pipeline.fit(X_train, y_train)
# Predict the training data
lm_test_predictions = lm_pipeline.predict(X_test)

In [213]:
# Now let's evaluate the model
import sklearn.metrics as metrics
MSE = metrics.mean_squared_error(y_test, lm_test_predictions)
print('MSE: ',MSE)
print('RMSE: ',np.sqrt(MSE))
R2 = metrics.r2_score(y_test, lm_test_predictions)
print('R2: ',R2)
adj_R2 = 1 - ( (1-R2)*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1) )
print('adj. R2: ',adj_R2)

MSE:  553.0969758245503
RMSE:  23.518013857988738
R2:  0.4063950701365835
adj. R2:  0.40502584455639723


In [215]:

lasso = Lasso(alpha =0.0005, random_state=125)

lasso_pipeline = make_pipeline(column_transform, lasso)

lasso_pipeline.fit(X_train, y_train)

X_lasso_predictions = lasso_pipeline.predict(X_test)

### Evaluating the model ###
print("Model performance for training set:")
MSE = metrics.mean_squared_error(y_test, X_lasso_predictions)
print('MSE: ',MSE)
print('RMSE: ',np.sqrt(MSE))
R2 = metrics.r2_score(y_test, X_lasso_predictions)
print('R2: ',R2)
adj_R2 = 1 - ( (1-R2)*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1) )
print('adj. R2: ',adj_R2)

Model performance for training set:
MSE:  553.1016981527072
RMSE:  23.518114255881724
R2:  0.4063900019525275
adj. R2:  0.4050207646819275
